In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
data = pd.read_csv("monte_carlo.csv")

In [3]:
class NashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        
        p = []
        
        for bound in [[0,0],[1,1],[0,1],[1,0]]:
            util1 = np.dot(self.exog[:,0:k], params[0:k])  + params[k]*bound[1]
            util2 = np.dot(self.exog[:,k:], params[k+1:2*k+1]) + params[2*k+1]*bound[0]
            prob = np.exp(util1)**bound[0]/( 1.+np.exp(util1) )
            prob = prob*np.exp(util2)**bound[1]/( 1.+np.exp(util2) )
            p.append(prob)

        p00 = (1 - self.endog[:,0])*(1 - self.endog[:,1])
        p11 = self.endog[:,0]*self.endog[:,1]
        p10 = self.endog[:,0]* (1 - self.endog[:,1])
        p01 = (1 - self.endog[:,0]) * self.endog[:,1]
        
        lambd = np.exp(params[6])/(1+np.exp(params[6]))        
        ll = p00 * p[0] + p11 *p[1]
        ll = ll + p01 * ( lambd * p[2] + (1-lambd)*(1 - p[0] - p[1] - p[3] ) )
        ll = ll + p10 * ( (1-lambd) *p[3] + lambd*(1 - p[0] - p[1] - p[2] ) )

        return -np.log(ll).sum()
        
        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+3)
        start_params = np.array([1, 4, -3 ,2, 3, -2 ,-1])
        return super(NashLogit, self).fit(start_params=start_params,**kwds)

        
        
N = data.shape[0]
y = data[['y1','y2']]
x = data[['x1','x2']]

x.insert(0,'x01',np.ones(N))
x.insert(2,'x02',np.ones(N))

model = NashLogit(y,x)
model = model.fit()
print(model.summary(xname=["b01","b11","d1","b02","b12","d2","rho"]))

                              NashLogit Results                               
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -615.99
Model:                      NashLogit   AIC:                             1238.
Method:            Maximum Likelihood   BIC:                             1253.
Date:                Fri, 05 Mar 2021                                         
Time:                        14:31:14                                         
No. Observations:                1000                                         
Df Residuals:                     997                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
b01            0.9698      0.190      5.111      0.000       0.598       1.342
b11            4.0206      0.287     13.991      0.0

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
